# Applied Data Science Capstone Project (Week 3)
## Segmenting and Clustering Neighborhoods in Toronto

In [22]:
# Import modules
import numpy as np
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cmap
import matplotlib.colors as colors
from sklearn.cluster import KMeans

## Read the table with pandas and load as a DataFrame

In [24]:
# Scrape data from Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_alldata = pd.read_html(url, header=0)[0]
df_alldata.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Remove Boroughs that are 'Not assigned'

In [26]:
#Remove Boroughs that are 'Not assigned'
df_canada = df_alldata[df_alldata.Borough != 'Not assigned'].reset_index(drop = True)
df_canada.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## If more than one neighborhood exist in one postal code area, combined them into one row with the neighborhoods separated with commas

## Once combined, remove the duplicates 

In [28]:
# combine neighborhoods in same postal code in one row with the neighborhoods separated with commas
df_canada["Neighbourhood"] = df_canada.groupby("Postal Code")["Neighbourhood"].transform(lambda ii: ', '.join(ii))

#remove duplicates
df_canada = df_canada.drop_duplicates()
df_canada.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough

In [30]:
# cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
if sum(df_canada["Neighbourhood"] == "Not assigned")==0:
    print("No neighborhood with \"Not assigned\" status")
else:
    for idx, row in df_canada.iterrows():
        if row["Neighbourhood"] == "Not assigned":
            df_canada.at[idx, "Neighbourhood"] == row["Borough"]
    print("Filled Neighborhood info")

No neighborhood with "Not assigned" status


## Print DataFrame shape

In [32]:
# print DataFrame shape
df_canada.shape

(103, 3)

# Get the latitude and the longitude coordinates of each neighborhood to use with FourSquare API

## get the geospatial Coordinates Data from https://cocl.us/Geospatial_data

In [38]:
# get the location data and load in a DataFrame
url='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_geocoord=pd.read_csv(url)
df_geocoord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge two dataframes based on 'PostalCode' 

In [41]:
df_canada = pd.merge(df_canada, df_geocoord, on = 'Postal Code')
df_canada.head()

,Postal Code,Borough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M3A,North York,Parkwoods,43.753259,-79.329656,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,43.662301,-79.389494
